<a href="https://colab.research.google.com/github/fahimku2020/fahimku2020/blob/main/Best_bart_summarizer_for_rare_words_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install  transformers
!pip install wikipedia
!pip install textwrap

ERROR: Could not find a version that satisfies the requirement textwrap (from versions: none)
ERROR: No matching distribution found for textwrap


In [18]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
import wikipedia
import textwrap

class BartWikipediaSummarizer:
    def __init__(self, model_name='facebook/bart-large-cnn'):
        """
        Initialize Bart summarization model and tokenizer

        Args:
            model_name (str): Pretrained Bart model for summarization
        """
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Load pre-trained Bart model and tokenizer
        self.tokenizer = BartTokenizer.from_pretrained(model_name)
        self.model = BartForConditionalGeneration.from_pretrained(model_name).to(self.device)

    def fetch_wikipedia_content(self, topic, sentences=10):
        """
        Fetch content from Wikipedia for a given topic

        Args:
            topic (str): Wikipedia search topic
            sentences (int): Number of sentences to extract

        Returns:
            str: Extracted Wikipedia content
        """
        try:
            # Search and get the Wikipedia page
            page = wikipedia.page(topic)

            # Extract content and limit to specified number of sentences
            content = page.summary
            return content

        except wikipedia.exceptions.DisambiguationError as e:
            print(f"Multiple matches found. Possible topics: {e.options}")
            return None

        except wikipedia.exceptions.PageError:
            print(f"No Wikipedia page found for topic: {topic}")
            return None

    def generate_summary(self, text, max_length=450, min_length=350):
        """
        Generate abstractive summary using Bart

        Args:
            text (str): Input text to summarize
            max_length (int): Maximum summary length
            min_length (int): Minimum summary length

        Returns:
            str: Generated summary
        """
        # Prepare input for Bart model
        inputs = self.tokenizer(
            text,
            max_length=1024,
            return_tensors='pt',
            truncation=True
        ).to(self.device)

        # Generate summary
        summary_ids = self.model.generate(
            inputs['input_ids'],
            num_beams=3,
            max_length=max_length,
            min_length=min_length,
            early_stopping=True
        )

        # Decode and return summary
        summary = self.tokenizer.decode(
            summary_ids[0],
            skip_special_tokens=True
        )

        return summary

    def summarize_wikipedia_topic(self, topic):
        """
        Fetch Wikipedia content and generate summary

        Args:
            topic (str): Wikipedia topic to summarize

        Returns:
            dict: Original content and generated summary
        """
        # Fetch Wikipedia content
        content = self.fetch_wikipedia_content(topic)

        if content:
            # Generate summary
            summary = self.generate_summary(content)

            return {
                'original_content': content,
                'summary': summary
            }

        return None

def main():
    # Example usage
    summarizer = BartWikipediaSummarizer()

    # List of topics to summarize
    topics = [
        'Amitab Bachan'
            ]

    for topic in topics:
        print(f"\n{'='*50}")
        print(f"SUMMARIZING: {topic}")
        print(f"{'='*50}")

        result = summarizer.summarize_wikipedia_topic(topic)

        if result:
            print("\nOriginal Content:")
            print(textwrap.fill(result['original_content'], width=100))

            print("\n\nAbstractive Summary:")
            print(textwrap.fill(result['summary'], width=100))

        print("\n")

if __name__ == '__main__':
    main()


SUMMARIZING: Amitab Bachan

Original Content:
Amitabh Bachchan (pronounced [əmɪˈt̪ɑːbʱ ˈbətːʃən] ; born Amitabh Srivastava; 11 October 1942) is an
Indian actor who works in Hindi cinema. He is often considered one of the greatest, most
accomplished and commercially successful actors in the history of Indian cinema. With a cinematic
career spanning over five decades, he has played pivotal roles in over 200 films. Bachchan is often
hailed as the Shahenshah of Bollywood, Sadi Ke Mahanayak (translated as "Greatest actor of the
century" in Hindi), Star of the Millennium, or simply Big B. His dominance in the Indian film
industry during the 1970s–80s led the French director François Truffaut to describe it as a "one-man
industry". He is a recipient of several accolades including six National Film Awards and sixteen
Filmfare Awards. Amitabh Bachchan was born in 1942 in Allahabad (now Prayagraj) to the Hindi poet
Harivansh Rai Bachchan and his wife, the social activist Teji Bachchan. He was e